In [144]:
import pandas as pd
import numpy as np

In [145]:
train=pd.read_csv('train_set.csv')
test=pd.read_csv('test_set.csv')

In [146]:
train.head(3).append(train.tail(3))

,Year,OGC,OG,DC,Dept,UC,Union,JF,Job,EI,Salaries,Overtime,H/D,YT,Total_Compensation
0,2015,4,Community Health,DPH,Public Health,250,"SEIU - Health Workers, Local 1021",Med Therapy & Auxiliary,Morgue Attendant,6725,12196,0,0.00,Calendar,16158
1,2013,4,Community Health,DPH,Public Health,39,"Stationary Engineers, Local 39",Journeyman Trade,Stationary Engineer,25058,74639,2820,12703.31,Fiscal,115784
2,2015,6,General Administration & Finance,ASR,Assessor/Recorder,21,"Prof & Tech Engineers - Miscellaneous, Local 21",Appraisal & Taxation,Senior Real Property Appraiser,46108,100554,0,12424.50,Calendar,144708
287833,2014,2,"Public Works, Transportation & Commerce",HHP,PUC Hetch Hetchy,6,"Electrical Workers, Local 6",Journeyman Trade,Electronic Maintenance Tech,38352,104451,0,13054.94,Calendar,164669
287834,2013,1,Public Protection,JUV,Juvenile Probation,250,"SEIU - Health Workers, Local 1021",Housekeeping & Laundry,Porter,29266,14425,0,4051.90,Calendar,19594
287835,2015,4,Community Health,DPH,Public Health,791,"SEIU - Staff and Per Diem Nurses, Local 1021",Nursing,Special Nurse,13906,105561,8945,7993.89,Fiscal,154197


In [147]:
train['Union'].fillna('n/a',inplace=True)
train['JF'].fillna('n/a',inplace=True)

In [148]:
test['Union'].fillna('n/a',inplace=True)
test['JF'].fillna('n/a',inplace=True)

In [149]:
train.isnull().sum().sum(),test.isnull().sum().sum()

(0, 0)

In [150]:
train['Union'].unique()

array(['SEIU - Health Workers, Local 1021',
       'Stationary Engineers, Local 39',
       'Prof & Tech Engineers - Miscellaneous, Local 21',
       "Police Officers' Association", 'SEIU - Miscellaneous, Local 1021',
       'SEIU - Human Services, Local 1021',
       'Automotive Machinists, Local 1414',
       'Firefighters - Chiefs/Fire Boat Workers, Local 798',
       'Firefighters - Miscellaneous, Local 798',
       'Transportation Workers, Local 200',
       'Transport Workers - Transit Operators, Local 250-A',
       'Laborers, Local 261',
       'Prof & Tech Engineers - Court Attorneys, Local 21',
       "Deputy Sheriffs' Association",
       'SEIU - Staff and Per Diem Nurses, Local 1021',
       'Transport Workers - Auto Svc Workers, Local 250-A',
       'Municipal Executive Association - Miscellaneous',
       'Teamsters - Supervising Nurses, Local 856',
       'Electrical Workers, Local 6',
       'Prof & Tech Engineers - Personnel, Local 21',
       'Plumbers and Pipefitters

In [143]:
train['Local_code']=train['Union'].str.split(',').str.get(1)
train['Local_code']=train['Local_code'].str.split(' ').str.get(2)
train['Local_code'].fillna((train['Local_code'].mode()),inplace=True)

In [151]:
train.head()

,Year,OGC,OG,DC,Dept,UC,Union,JF,Job,EI,Salaries,Overtime,H/D,YT,Total_Compensation
0,2015,4,Community Health,DPH,Public Health,250,"SEIU - Health Workers, Local 1021",Med Therapy & Auxiliary,Morgue Attendant,6725,12196,0,0.00,Calendar,16158
1,2013,4,Community Health,DPH,Public Health,39,"Stationary Engineers, Local 39",Journeyman Trade,Stationary Engineer,25058,74639,2820,12703.31,Fiscal,115784
2,2015,6,General Administration & Finance,ASR,Assessor/Recorder,21,"Prof & Tech Engineers - Miscellaneous, Local 21",Appraisal & Taxation,Senior Real Property Appraiser,46108,100554,0,12424.50,Calendar,144708
3,2016,1,Public Protection,POL,Police,911,Police Officers' Association,Police Services,Sergeant 3,33369,140164,52754,13043.87,Fiscal,242323
4,2013,2,"Public Works, Transportation & Commerce",HHP,PUC Hetch Hetchy,21,"Prof & Tech Engineers - Miscellaneous, Local 21",Information Systems,IS Engineer-Journey,28684,58813,0,7655.28,Calendar,82106


In [152]:
from sklearn.preprocessing import LabelEncoder
#le=LabelEncoder()
set=['OGC','OG','DC','Dept','Union','JF','YT']
for i in set:
    le = LabelEncoder()
    train[i] = le.fit_transform(train[i])
    train[i]=train[i]+1
    test[i] = le.fit_transform(test[i])
    test[i]=test[i]+1

In [153]:
train.head()

,Year,OGC,OG,DC,Dept,UC,Union,JF,Job,EI,Salaries,Overtime,H/D,YT,Total_Compensation
0,2015,4,1,20,46,250,58,29,Morgue Attendant,6725,12196,0,0.00,1,16158
1,2013,4,1,20,46,39,64,24,Stationary Engineer,25058,74639,2820,12703.31,2,115784
2,2015,6,3,6,6,21,53,7,Senior Real Property Appraiser,46108,100554,0,12424.50,1,144708
3,2016,1,6,40,43,911,48,36,Sergeant 3,33369,140164,52754,13043.87,2,242323
4,2013,2,7,29,39,21,53,23,IS Engineer-Journey,28684,58813,0,7655.28,1,82106


In [154]:
train['Job']=train['Job'].str.lower()
train['Job'].nunique()

1136

In [155]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from scipy.sparse import csr_matrix
from scipy import sparse

In [156]:
v_1 = TfidfVectorizer(ngram_range=(1,3),stop_words="english", analyzer='word')
typ_tr =v_1.fit_transform(train['Job'])
typ_ts =v_1.transform(test['Job'])


v_1c = TfidfVectorizer(ngram_range=(2,6),stop_words="english", analyzer='char')
typ_trc =v_1c.fit_transform(train['Job'])
typ_tsc =v_1c.transform(test['Job'])

In [157]:
dftrain=train.drop(['Job','Total_Compensation'],axis=1)
dftest=test.drop(['Job'],axis=1)

In [158]:
dftest.shape

(47860, 13)

In [159]:
final_train = sparse.hstack((dftrain,typ_tr,typ_trc)).tocsr()
final_test = sparse.hstack((dftest,typ_ts ,typ_tsc)).tocsr()

In [160]:
target = (train['Total_Compensation'])

In [161]:
from sklearn.model_selection import train_test_split
X_trn, X_val, y_trn, y_val = train_test_split(final_train, target, test_size=0.1, random_state=121)
X_test = final_test

In [162]:
from sklearn.metrics import mean_squared_error

In [163]:
from lightgbm import LGBMRegressor
X=final_train
y=target

y_pred_tot=[]
err=[]
feature_importance_df = pd.DataFrame()

#For Stacking
lgbm_train_res = pd.DataFrame()
lgbm_test_res = pd.DataFrame()

from sklearn.model_selection import KFold,StratifiedKFold
fold=KFold(n_splits=4,shuffle=True,random_state=1996)
i=1
for train_index, test_index in fold.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    depth=10
    m=LGBMRegressor(n_estimators=2000,n_jobs=-1,learning_rate=0.005,max_depth=depth,num_leaves=2**depth-1)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=200,eval_metric="rmse")
    preds=m.predict(X_test,num_iteration=m.best_iteration_)
    #print(f"RMSLE is:",rmsle_lgb(y_test, preds))
    print(f"RMSE is: {np.sqrt(mean_squared_error((y_test), (preds)))}")
    err.append(np.sqrt(mean_squared_error((y_test), (preds))))
    p = m.predict(final_test)
    y_pred_tot.append(p)
    
    lgbm_train_res["iteration_"+str(i)] = m.predict(X)
    lgbm_test_res["iteration_"+str(i)] = m.predict(final_test)
    i=i+1

Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 25586.3	training's l2: 6.54657e+08	valid_1's rmse: 25762.7	valid_1's l2: 6.63718e+08
[400]	training's rmse: 10986.9	training's l2: 1.20711e+08	valid_1's rmse: 11259.2	valid_1's l2: 1.26771e+08
[600]	training's rmse: 6765.29	training's l2: 4.57692e+07	valid_1's rmse: 7234.59	valid_1's l2: 5.23392e+07
[800]	training's rmse: 5814.48	training's l2: 3.38081e+07	valid_1's rmse: 6414.97	valid_1's l2: 4.11519e+07
[1000]	training's rmse: 5562.93	training's l2: 3.09462e+07	valid_1's rmse: 6240.87	valid_1's l2: 3.89484e+07
[1200]	training's rmse: 5444.95	training's l2: 2.96474e+07	valid_1's rmse: 6188.12	valid_1's l2: 3.82928e+07
[1400]	training's rmse: 5381.74	training's l2: 2.89632e+07	valid_1's rmse: 6160.96	valid_1's l2: 3.79575e+07
[1600]	training's rmse: 5335.41	training's l2: 2.84666e+07	valid_1's rmse: 6143.32	valid_1's l2: 3.77404e+07
[1800]	training's rmse: 5305.75	training's l2: 2.8151e+07	valid_1's rm

In [164]:
preds=np.mean(y_pred_tot,0)

In [165]:
sub=pd.DataFrame({'Compensation':preds})
sub.to_csv('sub.csv',index=False)

In [166]:
test.shape,preds.shape

((47860, 14), (47860,))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_trn,y_trn)